In [1]:
import pynvml
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
info = pynvml.nvmlDeviceGetMemoryInfo(handle)
print(f'Total memory: {info.total / 1024**3:.2f} GB')
print(f'Free memory: {info.free / 1024**3:.2f} GB')
print(f'Used memory: {info.used / 1024**3:.2f} GB')
import dask_cudf
import cupy as cp
import rmm
import cudf
import gc


Total memory: 8.00 GB
Free memory: 7.60 GB
Used memory: 0.40 GB


In [2]:
import cudf.pandas
cudf.pandas.install()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
taxi_busyness_data= dask_cudf.read_csv('2020_Yellow_Taxi_Trip_Data_20240522.csv', blocksize="20MB")
info = pynvml.nvmlDeviceGetMemoryInfo(handle)

print(f'Used memory: {info.used / 1024**3:.2f} GB')

Used memory: 0.52 GB


In [3]:
taxi_busyness_data.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge'],
      dtype='object')

In [4]:
taxi_busyness_data = taxi_busyness_data[["tpep_pickup_datetime","tpep_dropoff_datetime","PULocationID","DOLocationID","passenger_count"]]
print(taxi_busyness_data.columns)
rows = taxi_busyness_data.shape[0].compute()
print(rows)
print(f'Used memory: {info.used / 1024**3:.2f} GB')

Index(['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID',
       'DOLocationID', 'passenger_count'],
      dtype='object')
24648499
Used memory: 0.52 GB


In [5]:
null_counts = taxi_busyness_data.isnull().sum()
print(null_counts.compute())

# for column in taxi_busyness_data.columns:
#     print(f"Value counts for {column}:")
#     value_counts = taxi_busyness_data[column].value_counts().compute()
#     print(value_counts)
#     print("\n")
#     # print(f"Unique values for {column}:")
    
#     # Replace null values with a placeholder or drop them
#     # taxi_busyness_data[column] = taxi_busyness_data[column].fillna(-1)  # Adjust based on data type and context
#     # print("hi")
    
#     # unique_values = taxi_busyness_data[column].unique_count()
#     # print(unique_values)
#     # print("\n")

# taxi_busyness_data.nunique().compute()
# info = pynvml.nvmlDeviceGetMemoryInfo(handle)

# print(f'Used memory: {info.used / 1024**3:.2f} GB')
# gc.collect()

tpep_pickup_datetime          0
tpep_dropoff_datetime         0
PULocationID                  0
DOLocationID                  0
passenger_count          809568
dtype: int64


In [6]:
for column in taxi_busyness_data.columns:
    print(taxi_busyness_data.groupby(column).size().compute())

tpep_pickup_datetime
01/01/2020 03:21:54 AM    1
01/01/2020 03:21:54 PM    1
01/01/2020 03:21:55 AM    3
01/01/2020 03:21:55 PM    1
01/01/2020 03:21:56 AM    4
                         ..
12/31/2020 10:36:08 AM    1
12/31/2020 10:36:10 PM    1
12/31/2020 09:12:57 PM    1
12/31/2020 09:12:58 PM    1
12/31/2020 10:36:11 PM    2
Length: 11775710, dtype: int64
tpep_dropoff_datetime
01/01/2020 03:38:48 PM    1
01/01/2020 03:38:49 AM    5
01/01/2020 03:38:49 PM    3
01/01/2020 03:38:50 AM    2
01/01/2020 03:38:50 PM    3
                         ..
12/31/2020 11:27:31 AM    1
12/31/2020 11:27:32 AM    1
12/31/2020 11:27:34 AM    3
12/31/2020 11:42:29 AM    1
12/31/2020 12:49:11 PM    4
Length: 11776107, dtype: int64
PULocationID
55      3810
58       163
60      1911
61     16356
62      4600
       ...  
120      253
121     2962
122     1328
123     2746
124     1767
Length: 262, dtype: int64
DOLocationID
50     282743
51       9921
52      17797
53       3213
54       5589
        ...  


In [7]:
taxi_busyness_data.head(5)

,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID,passenger_count
0,01/01/2020 12:28:15 AM,01/01/2020 12:33:03 AM,238,239,1
1,01/01/2020 12:35:39 AM,01/01/2020 12:43:04 AM,239,238,1
2,01/01/2020 12:47:41 AM,01/01/2020 12:53:52 AM,238,238,1
3,01/01/2020 12:55:23 AM,01/01/2020 01:00:14 AM,238,151,1
4,01/01/2020 12:01:58 AM,01/01/2020 12:04:16 AM,193,193,1


In [8]:
print(cudf.__version__)

24.04.01


In [9]:
pandas_dol=taxi_busyness_data["DOLocationID"].compute().to_pandas()
pandas_pul=taxi_busyness_data["PULocationID"].compute().to_pandas()
unique_in_pul = pandas_pul[~pandas_pul.isin(pandas_dol)].unique()
print(pandas_pul.shape,pandas_dol.shape)
# Find unique elements in pandas_dol that are not in pandas_pul
unique_in_dol = pandas_dol[~pandas_dol.isin(pandas_pul)].unique()
print(unique_in_pul)

print(unique_in_pul.shape,unique_in_dol.shape)
print("Unique elements in pandas_pul but not in pandas_dol:", unique_in_pul)
print("Unique elements in pandas_dol but not in pandas_pul:", unique_in_dol)

(24648499,) (24648499,)
[]
(0,) (1,)
Unique elements in pandas_pul but not in pandas_dol: []
Unique elements in pandas_dol but not in pandas_pul: [104]


In [10]:

# Compute the Dask cuDF DataFrame to bring it into memory as a cuDF DataFrame
computed_df = taxi_busyness_data.compute()

# Access the first element in the 'tpep_dropoff_datetime' column
first_element = computed_df["tpep_dropoff_datetime"].iloc[0]
print(first_element)
# Get the type of the first element
element_type = type(first_element)

print(f"Type of the first element in 'tpep_dropoff_datetime': {element_type}")
info = pynvml.nvmlDeviceGetMemoryInfo(handle)

print(f'Used memory: {info.used / 1024**3:.2f} GB')
gc.collect()

01/01/2020 12:33:03 AM
Type of the first element in 'tpep_dropoff_datetime': <class 'str'>
Used memory: 5.18 GB


13975

In [11]:
taxi_filled_na = taxi_busyness_data.fillna({"passenger_count": 1})

type(taxi_filled_na)


dask_cudf.core.DataFrame

In [12]:
max_value = taxi_filled_na['passenger_count'].max().compute()
print("Maximum value in column 'passenger_count':", max_value)

taxi_filled_na.head(5)

Maximum value in column 'passenger_count': 9


,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID,passenger_count
0,01/01/2020 12:28:15 AM,01/01/2020 12:33:03 AM,238,239,1
1,01/01/2020 12:35:39 AM,01/01/2020 12:43:04 AM,239,238,1
2,01/01/2020 12:47:41 AM,01/01/2020 12:53:52 AM,238,238,1
3,01/01/2020 12:55:23 AM,01/01/2020 01:00:14 AM,238,151,1
4,01/01/2020 12:01:58 AM,01/01/2020 12:04:16 AM,193,193,1


In [13]:
meta = {
    'tpep_pickup_datetime': 'datetime64[ms]',
    'tpep_dropoff_datetime': 'datetime64[ms]',
    'PULocationID': 'int16',
    'DOLocationID': 'int16',
    'passenger_count': 'int16'
}
meta_data = taxi_filled_na.map_partitions(lambda df: df.head(), meta=meta)

In [14]:
result  = meta_data.compute()

In [15]:
result.dtypes

tpep_pickup_datetime     object
tpep_dropoff_datetime    object
PULocationID              int64
DOLocationID              int64
passenger_count           int64
dtype: object

In [16]:
result['tpep_pickup_datetime'] = cudf.to_datetime(result['tpep_pickup_datetime'], format='%m/%d/%Y %I:%M:%S %p')
result['tpep_dropoff_datetime'] = cudf.to_datetime(result['tpep_dropoff_datetime'], format='%m/%d/%Y %I:%M:%S %p')

In [17]:
result['tpep_pickup_datetime'] = result['tpep_pickup_datetime'].dt.floor('S')
result['tpep_dropoff_datetime'] = result['tpep_dropoff_datetime'].dt.floor('S')


/home/eamonn-walsh/miniconda3/envs/rapids-24.04/lib/python3.11/contextlib.py:81: FutureWarning: FutureWarning: S is deprecated and will be removed in a future version, please use s instead.
  return func(*args, **kwds)


In [18]:
final = result.copy()
final['tpep_pickup_datetime'] = final['tpep_pickup_datetime'].dt.strftime('%Y-%m-%dT%H:%M:%S')
final['tpep_dropoff_datetime'] = final['tpep_dropoff_datetime'].dt.strftime('%Y-%m-%dT%H:%M:%S')

In [19]:
result.dtypes

tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
PULocationID                      int64
DOLocationID                      int64
passenger_count                   int64
dtype: object

In [20]:
result['pickup_hour'] = result['tpep_pickup_datetime'].dt.hour
result['pickup_minute'] = result['tpep_pickup_datetime'].dt.minute
result['pickup_second'] = result['tpep_pickup_datetime'].dt.second
result['pickup_time'] = (result['pickup_hour'].astype(str).str.zfill(2) + ':' +
                         result['pickup_minute'].astype(str).str.zfill(2) + ':' +
                         result['pickup_second'].astype(str).str.zfill(2))
result['dropoff_hour'] = result['tpep_dropoff_datetime'].dt.hour
result['dropoff_minute'] = result['tpep_dropoff_datetime'].dt.minute
result['dropoff_second'] = result['tpep_dropoff_datetime'].dt.second
result['dropoff_time'] = (result['dropoff_hour'].astype(str).str.zfill(2) + ':' +
                         result['dropoff_minute'].astype(str).str.zfill(2) + ':' +
                         result['dropoff_second'].astype(str).str.zfill(2))



In [21]:
result.dtypes

tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
PULocationID                      int64
DOLocationID                      int64
passenger_count                   int64
pickup_hour                       int16
pickup_minute                     int16
pickup_second                     int16
pickup_time                      object
dropoff_hour                      int16
dropoff_minute                    int16
dropoff_second                    int16
dropoff_time                     object
dtype: object

In [22]:
print(result.head())

  tpep_pickup_datetime tpep_dropoff_datetime  PULocationID  DOLocationID  \
0  2020-01-01 00:28:15   2020-01-01 00:33:03           238           239   
1  2020-01-01 00:35:39   2020-01-01 00:43:04           239           238   
2  2020-01-01 00:47:41   2020-01-01 00:53:52           238           238   
3  2020-01-01 00:55:23   2020-01-01 01:00:14           238           151   
4  2020-01-01 00:01:58   2020-01-01 00:04:16           193           193   

   passenger_count  pickup_hour  pickup_minute  pickup_second pickup_time  \
0                1            0             28             15    00:28:15   
1                1            0             35             39    00:35:39   
2                1            0             47             41    00:47:41   
3                1            0             55             23    00:55:23   
4                1            0              1             58    00:01:58   

   dropoff_hour  dropoff_minute  dropoff_second dropoff_time  
0             0  

In [23]:
max_value = result['pickup_hour'].max()
print("Max value of column 'pickup_hour':", max_value)
max_value = result['pickup_minute'].max()
print("Max value of column 'pickup_minute':", max_value)
max_value = result['pickup_second'].max()
print("Max value of column 'pickup_second':", max_value)

Max value of column 'pickup_hour': 23
Max value of column 'pickup_minute': 59
Max value of column 'pickup_second': 59


In [24]:
max_value = result['dropoff_hour'].max()
print("Max value of column 'dropoff_hour':", max_value)
max_value = result['dropoff_minute'].max()
print("Max value of column 'dropoff_minute':", max_value)
max_value = result['dropoff_second'].max()
print("Max value of column 'dropoff_second':", max_value)

Max value of column 'dropoff_hour': 23
Max value of column 'dropoff_minute': 59
Max value of column 'dropoff_second': 59


In [25]:
null_counts = taxi_filled_na.isnull().sum()
print(null_counts.compute())

tpep_pickup_datetime     0
tpep_dropoff_datetime    0
PULocationID             0
DOLocationID             0
passenger_count          0
dtype: int64


In [26]:
final.to_csv('saved_csvs/taxi_trip.csv', index=False)

In [27]:
import plotly.express as px

In [28]:
# Convert cudf DataFrame to pandas DataFrame
final_pandas = result.to_pandas()

# Create a new column for the 30-minute intervals manually using pandas
final_pandas['30min_interval'] = final_pandas['tpep_dropoff_datetime'].dt.floor('30T')

# Extract the hour and minute to create a categorical interval
final_pandas['hour_minute'] = final_pandas['30min_interval'].dt.strftime('%H:%M')

# Convert the pandas DataFrame back to cudf DataFrame
final = cudf.DataFrame.from_pandas(final_pandas)

# Group by the 30-minute intervals and count the number of drop-offs
grouped = final.groupby('hour_minute').size().reset_index(name='number_of_dropoffs')

# Convert cuDF DataFrame to pandas DataFrame for use with Plotly
grouped_pandas = grouped.to_pandas()
print(grouped_pandas)
# Create a violin plot using Plotly
fig = px.violin(grouped_pandas, y='number_of_dropoffs', x='hour_minute',
                labels={'hour_minute': '30-Minute Interval', 'number_of_dropoffs': 'Number of Drop-offs'},
                title='Distribution of Drop-offs per 30-Minute Interval')

# Display the plot
fig.show()

/tmp/ipykernel_4487/3385554364.py:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  final_pandas['30min_interval'] = final_pandas['tpep_dropoff_datetime'].dt.floor('30T')


   hour_minute  number_of_dropoffs
0        00:00                   9
1        00:30                   8
2        01:00                   5
3        01:30                   6
4        02:00                   2
5        02:30                   2
6        03:00                   2
7        04:00                   1
8        04:30                   2
9        05:00                   2
10       05:30                  12
11       06:00                   5
12       06:30                   9
13       07:00                   1
14       07:30                   8
15       08:00                   6
16       08:30                  19
17       09:00                  21
18       09:30                  19
19       10:00                  10
20       10:30                  22
21       11:00                  14
22       11:30                  13
23       12:00                  12
24       12:30                  16
25       13:00                  21
26       13:30                  14
27       14:00      